In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sensitivity_tests import *
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import os

from gurobipy import *
setParam("OutputFlag", 0)

Academic license - for non-commercial use only


In [ ]:
ELO_var = [10, 20]
n_instances = 1000
n_trials = 250
n_items = 15

dataset = []
for var in tqdm(ELO_var):
    data_source = SynthELOTournamentSource(n_items, n_games=1, comp_var=var)
    for instance_idx in range(n_instances):
        dataset.append(ProblemInstance(data_source).collect_data(n_sensitivity_trials=n_trials))
df = pd.DataFrame(dataset)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
df.to_csv("sensitivity_dataset_binary.csv")

In [ ]:
sensitivities = [col for col in df.columns if "mean_sensitivity" in col]
sensitivities

In [ ]:
df["overall_mean_sensitivity"] = df[sensitivities].mean(axis=1)

In [ ]:
df.plot.scatter("p_lowerbound", "overall_mean_sensitivity", title="Sensitivity by p")

In [ ]:
df.plot.scatter("kendall_w", "overall_mean_sensitivity", title="Sensitivity by Kendall W")

In [ ]:
df.plot.scatter("max_L2_dist", "overall_mean_sensitivity", title="Sensitivity by Max L2 Dist")

In [ ]:
df.plot.scatter("mean_L2_dist", "overall_mean_sensitivity", title="Sensitivity by Mean L2 Dist")

In [ ]:
df.plot.scatter("min_tau", "overall_mean_sensitivity", title="Sensitivity by Min Tau")

In [ ]:
df.plot.scatter("mean_tau", "overall_mean_sensitivity", title="Sensitivity by Mean Tau")

In [ ]:
def plot_taus(rankability_vals, taus, method_name, scatter_alpha=0.01, histogram_bins=30, save_dir="rankability_figures_run5"):
    # Create directory for saving if specified
    saving = False
    if save_dir is not None and save_dir != "":
        Path(save_dir).mkdir(parents=True, exist_ok=True)
        saving = True
    
    # Plot scatter plot
    plt.scatter(rankability_vals, taus, alpha=scatter_alpha)
    title = "%s Sensitivity Scatter" % method_name
    plt.title(title)
    plt.xlabel('Rankability')
    plt.xlim(0.0, 1.0)
    plt.ylabel('Tau')
    plt.ylim(-1.0, 1.0)
    if saving:
        save_path = os.path.join(save_dir, title.replace(" ", "_") + ".png")
        plt.savefig(save_path)
    plt.show()
    
    # Plot 2D Histogram
    plt.hist2d(rankability_vals, taus, bins=histogram_bins, range=[[0.0, 1.0], [-1.0, 1.0]])
    title = "%s Sensitivity Histogram" % method_name
    plt.title(title)
    plt.xlabel('Rankability')
    plt.ylabel('Tau')
    if saving:
        save_path = os.path.join(save_dir, title.replace(" ", "_") + ".png")
        plt.savefig(save_path)
    plt.show()